In [3]:
# ============================================================
# Google Colab environment setup (pinned versions)
# ============================================================

import sys
import os
import subprocess

if "google.colab" in sys.modules:
    print("Running in Google Colab")
    print("Python version:", sys.version.split()[0])

    # ---- Required package versions --------------------------
    requirements = {
        "numpy": "2.4.0",
        "scipy": "1.16.3",
        "matplotlib": "3.10.8",
        "pandas": "2.3.3",
    }

    # ---- Check currently loaded versions --------------------
    restart_needed = False

    for pkg, required_version in requirements.items():
        try:
            module = __import__(pkg)
            installed_version = module.__version__
        except Exception:
            installed_version = None

        print(f"{pkg}: {installed_version} (required: {required_version})")

        if installed_version != required_version:
            restart_needed = True

    # ---- Install if needed ----------------------------------
    if restart_needed:
        print("\nInstalling pinned package versions...")

        pip_args = [
            f"{pkg}=={ver}" for pkg, ver in requirements.items()
        ]

        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", *pip_args]
        )

        print("Installation complete.")
        print("Restarting runtime to load correct packages...")

        # This will appear as a "crash" in Colab — expected behavior
        os.kill(os.getpid(), 9)

    else:
        print("\nAll required package versions already installed.")

else:
    print("Not running in Google Colab — setup skipped.")
    print("Python version:", sys.version.split()[0])

Not running in Google Colab — setup skipped.
Python version: 3.12.12


In [ ]:
# --- Version check ---
import numpy
import scipy
import matplotlib
import pandas

print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("pandas:", pandas.__version__)

In [2]:
"""
CEE6501 — Bandwidth / Sparsity timing demo

This script compares THREE approaches on matrices that represent the SAME
structural graph (same physics), but with DIFFERENT node numberings
(low vs high bandwidth):

1) Dense solve (NumPy): np.linalg.solve
   - ignores sparsity and bandwidth completely

2) Sparse LU without reordering:
   - splu(permc_spec="NATURAL")

3) Sparse LU with fill-reducing reordering:
   - splu(permc_spec="COLAMD")

Key idea:
- Bandwidth is an artifact of DOF ordering
- Sparse solvers can exploit (or fix) bad orderings
- Dense solvers cannot
"""

import time
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla


# =============================================================================
# Utilities
# =============================================================================

def median_time(fn, repeats=7, warmup=1):
    """Return median runtime (seconds)."""
    for _ in range(warmup):
        fn()
    ts = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        fn()
        ts.append(time.perf_counter() - t0)
    return float(np.median(ts))


def half_bandwidth_from_sparse(K: sp.spmatrix) -> int:
    """Half-bandwidth b = 1 + max|i-j| over nonzeros of K."""
    K = K.tocoo()
    if K.nnz == 0:
        return 0
    return int(1 + np.max(np.abs(K.row - K.col)))


def permute_sparse(K: sp.spmatrix, p: np.ndarray) -> sp.spmatrix:
    """Return P K P^T using permutation vector p."""
    return K.tocsr()[p, :][:, p].tocsc()


# =============================================================================
# Build an SPD sparse matrix with realistic structure
# =============================================================================

def make_grid_graph_spd(nx: int, ny: int, seed: int = 0, diag_boost: float = 1.0) -> sp.csc_matrix:
    """
    Build an SPD matrix from a 2D grid graph using a weighted Laplacian:
        K = L + diag_boost * I

    This mimics a stiffness matrix:
    - symmetric
    - sparse
    - SPD once constrained (diag_boost > 0)
    """
    rng = np.random.default_rng(seed)
    n = nx * ny

    def idx(i, j):
        return i * ny + j

    rows, cols, vals = [], [], []
    diag = np.zeros(n)

    for i in range(nx):
        for j in range(ny):
            u = idx(i, j)

            if i + 1 < nx:
                v = idx(i + 1, j)
                w = 0.5 + rng.random()
                rows += [u, v]
                cols += [v, u]
                vals += [-w, -w]
                diag[u] += w
                diag[v] += w

            if j + 1 < ny:
                v = idx(i, j + 1)
                w = 0.5 + rng.random()
                rows += [u, v]
                cols += [v, u]
                vals += [-w, -w]
                diag[u] += w
                diag[v] += w

    rows += list(range(n))
    cols += list(range(n))
    vals += list(diag + diag_boost)

    return sp.coo_matrix((vals, (rows, cols)), shape=(n, n)).tocsc()


# =============================================================================
# Node orderings (same physics, different bandwidth)
# =============================================================================

def permutation_low_bandwidth(nx: int, ny: int) -> np.ndarray:
    """Natural grid ordering (relatively low bandwidth)."""
    return np.arange(nx * ny, dtype=int)


def permutation_high_bandwidth(nx: int, ny: int) -> np.ndarray:
    """Deliberately bad ordering to inflate bandwidth."""
    p = np.arange(nx * ny)
    return np.concatenate([p[p % 2 == 0], p[p % 2 == 1]])


# =============================================================================
# Solvers
# =============================================================================

def time_dense_solve(Ks: sp.spmatrix, nrhs: int = 1, repeats: int = 5) -> float:
    """Dense solve using NumPy (ignores sparsity/bandwidth)."""
    Kd = Ks.toarray()
    n = Kd.shape[0]
    rng = np.random.default_rng(123)
    B = rng.normal(size=(n, nrhs))

    def run():
        _ = np.linalg.solve(Kd, B)

    return median_time(run, repeats=repeats, warmup=1)


def time_sparse_lu(Ks: sp.spmatrix, permc_spec: str, nrhs: int = 1, repeats: int = 7):
    """
    Sparse LU using SuperLU.
    permc_spec:
      - "NATURAL"  : no fill-reducing reordering
      - "COLAMD"   : fill-reducing reordering
    """
    Ks = Ks.tocsc()
    n = Ks.shape[0]
    rng = np.random.default_rng(123)
    B = rng.normal(size=(n, nrhs))

    # factorization timing
    t0 = time.perf_counter()
    lu = spla.splu(Ks, permc_spec=permc_spec)
    t_factor = time.perf_counter() - t0

    def run():
        _ = lu.solve(B)

    t_solve = median_time(run, repeats=repeats, warmup=1)
    return float(t_factor), float(t_solve)


# =============================================================================
# Main experiment
# =============================================================================

if __name__ == "__main__":
    # Grid size (n = nx * ny)
    nx, ny = 40, 40      # n = 2025
    n = nx * ny

    K_base = make_grid_graph_spd(nx, ny, seed=0, diag_boost=1.0)

    p_low  = permutation_low_bandwidth(nx, ny)
    p_high = permutation_high_bandwidth(nx, ny)

    K_low  = permute_sparse(K_base, p_low)
    K_high = permute_sparse(K_base, p_high)

    b_low  = half_bandwidth_from_sparse(K_low)
    b_high = half_bandwidth_from_sparse(K_high)

    print("=== Matrix info ===")
    print(f"n = {n}")
    print(f"nnz(K) = {K_base.nnz}")
    print(f"half-bandwidth (low ordering)  = {b_low}")
    print(f"half-bandwidth (high ordering) = {b_high}")
    print()

    # 1) Dense baseline
    print("=== Dense solve (NumPy) ===")
    t_dense_low  = time_dense_solve(K_low)
    t_dense_high = time_dense_solve(K_high)
    print(f"dense solve (low bw):  {t_dense_low:.4f} s")
    print(f"dense solve (high bw): {t_dense_high:.4f} s")
    print("(dense ignores sparsity and bandwidth)\n")

    # 2) Sparse LU, no reordering
    print('=== Sparse LU — permc_spec="NATURAL" ===')
    tf_nat_low,  ts_nat_low  = time_sparse_lu(K_low,  "NATURAL")
    tf_nat_high, ts_nat_high = time_sparse_lu(K_high, "NATURAL")
    print(f"factor (low bw):  {tf_nat_low:.4f} s   solve: {ts_nat_low:.4f} s")
    print(f"factor (high bw): {tf_nat_high:.4f} s   solve: {ts_nat_high:.4f} s")
    print(f"factor slowdown (high/low): {tf_nat_high / tf_nat_low:.2f}×\n")

    # 3) Sparse LU, with reordering
    print('=== Sparse LU — permc_spec="COLAMD" ===')
    tf_col_low,  ts_col_low  = time_sparse_lu(K_low,  "COLAMD")
    tf_col_high, ts_col_high = time_sparse_lu(K_high, "COLAMD")
    print(f"factor (low bw):  {tf_col_low:.4f} s   solve: {ts_col_low:.4f} s")
    print(f"factor (high bw): {tf_col_high:.4f} s   solve: {ts_col_high:.4f} s")
    print(f"factor slowdown (high/low): {tf_col_high / tf_col_low:.2f}×\n")

    print("=== Key takeaways ===")
    print("- Dense solvers ignore sparsity and bandwidth.")
    print("- Sparse solvers are sensitive to DOF ordering.")
    print("- Fill-reducing reordering largely neutralizes bad numbering.")
    print("- This is why structural analysis software reorders DOFs internally.")


=== Matrix info ===
n = 1600
nnz(K) = 7840
half-bandwidth (low ordering)  = 41
half-bandwidth (high ordering) = 801

=== Dense solve (NumPy) ===
dense solve (low bw):  0.0161 s
dense solve (high bw): 0.0153 s
(dense ignores sparsity and bandwidth)

=== Sparse LU — permc_spec="NATURAL" ===
factor (low bw):  0.0025 s   solve: 0.0001 s
factor (high bw): 0.1153 s   solve: 0.0004 s
factor slowdown (high/low): 45.46×

=== Sparse LU — permc_spec="COLAMD" ===
factor (low bw):  0.0020 s   solve: 0.0001 s
factor (high bw): 0.0021 s   solve: 0.0001 s
factor slowdown (high/low): 1.04×

=== Key takeaways ===
- Dense solvers ignore sparsity and bandwidth.
- Sparse solvers are sensitive to DOF ordering.
- Fill-reducing reordering largely neutralizes bad numbering.
- This is why structural analysis software reorders DOFs internally.
